# Лабораторная работа №3
## Выполнил студент группы Мосева Алеся Сергеевна БВТ2002

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [2]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict
import time

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [11]:
def prefix_function(text):
    arr = [0] * len(text)
    for i in range(1, len(text)):
        j = arr[i-1]
        while j > 0 and text[i] != text[j]:
            j = arr[j-1]
        if text[i] == text[j]:
            j+=1
        arr[i] = j

def kmp_search(text, sub_text, start_index=0, reg=True):
    if len(sub_text) > len(text):
        return 'error'
    if start_index != 0:
        text = text[start_index:]
    if reg==False:
        text = text.lower()
        sub_text = sub_text.lower()
    p = prefix_function(sub_text)
    j = 0
    for i in range(start_index, len(text)):
        while j > 0 and text[i] != sub_text[j]:
            j = p[j-1]
        if text[i] == sub_text[j]:
            j += 1
        if j >= len(sub_text):
            return i - j + 1
    return None

text = 'персональные данные'
sub_text = 'данные'
start_time = time.time()
print(kmp_search(text, sub_text), (time.time()-start_time)*1000)

13 0.053882598876953125


#### Упрощенный алгоритм Бойера-Мура

In [3]:
t = "данные"

start_time = time.time()

S = set()
M = len(t) 
d = {}

for i in range(M-2, -1, -1):
    if t[i] not in S:    
        d[t[i]] = M-i-1
        S.add(t[i])

if t[M-1] not in S:
    d[t[M-1]] = M

d['*'] = M


a = "персональные данные"
N = len(a)

if N >= M:
    i = M-1 

    while(i < N):
        k = 0
        j = 0
        flBreak = False
        for j in range(M-1, -1, -1):
            if a[i-k] != t[j]:
                if j == M-1:
                    off = d[a[i]] if d.get(a[i], False) else d['*'] 
                else:
                    off = d[t[j]] 

                i += off
                flBreak = True
                break

            k += 1

        if not flBreak:
            print(f"{i-k+1}", (time.time()-start_time)*1000)
            break
    else:
        print("образ не найден")
else:
    print("образ не найден")

13 1.0008811950683594


In [13]:
text = 'персональные данные'
sub_text = 'данные'
start_time = time.time()
print(text.find(sub_text), (time.time() - start_time)*1000)

13 0.04291534423828125


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [4]:
from queue import PriorityQueue

N = 4

def moves(position):
    blank = position.index(0)
    i, j = divmod(blank, N)
    offsets = []
    if i > 0: offsets.append(-N)     
    if i < N - 1: offsets.append(N) 
    if j > 0: offsets.append(-1)     
    if j < N - 1: offsets.append(1)  
    for offset in offsets:
        swap = blank + offset
        yield tuple(position[swap] if x==blank else position[blank] if x==swap else position[x] for x in range(N*N))

def parity(permutation):
    count = 0
    for i in range(14):
        for j in range(i + 1, 15):
            if permutation[i] > permutation[j]:
                count += 1
    return count % 2


class Position:
    def __init__(self, position, start_distance):
        self.position = position
        self.start_distance = start_distance

    def __lt__(self, other):
        return self.start_distance < other.start_distance

    def __str__(self):
        return '\n'.join((N*'{:3}').format(*[i%(N*N) for i in self.position[i:]]) for i in range(0, N*N, N))

SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
start = (1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0)

path = []
if parity(start):
    print(path)
else:
    p = Position(start, 0)

    print(p, "\n")

    candidates = PriorityQueue()
    candidates.put(p)

    visited = {p}

    came_from = {p.position: None}
    
    while p.position != SOLVED:
        p = candidates.get()
        for k in moves(p.position):
            if k not in visited:
                candidates.put(Position(k, p.start_distance + 1))
                came_from[k] = p
                visited.add(k)

    prev = p
    while p.position != start:
        p = came_from[p.position]
        number = p.position[prev.position.index(0)]
        path.append(number)
        prev = p
    path.reverse()

    print(path)

  1  2  3  4
  5  6  7  8
 13  9 11 12
 10 14 15  0 

[15, 14, 10, 13, 9, 10, 14, 15]


### Вывод

Время для Кнутта-Мориса-Пратта: 0,05
Время для Бойера-Мура: 1,0
Время для стандартной сортировки: 0,04